<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/machine-learning-bootcamp/unsupervised/03_association_rules/02_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn
```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję `0.22.1`

### Spis treści:
1. [Import bibliotek](#0)
2. [Załadownaie danych](#1)
3. [Przygotowanie danych](#2)
4. [Kodowanie transakcji](#3)
5. [Algorytm Apriori](#4)




### <a name='0'></a> Import bibliotek

In [1]:
print("############################################################")
print("Etap12. Odcinek: Reguły asocjacyjne - algorytm Apriori - ćwiczenie 2")

import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

############################################################
Etap12. Odcinek: Reguły asocjacyjne - algorytm Apriori - ćwiczenie 2


### <a name='1'></a> Załadownaie danych

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2021-01-15 18:38:05--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.128, 74.125.69.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  --.-KB/s    in 0.02s   

2021-01-15 18:38:07 (104 MB/s) - ‘products.csv’ saved [2166953/2166953]

--2021-01-15 18:38:07--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-str

In [3]:
# używa dwóch spośród 4 kolumn
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [4]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


### <a name='2'></a> Przygotowanie danych

In [5]:
# kolumna z lewej strony to indeksy wierszy
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head()

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado


In [6]:
data.describe()

,order_id,product_id
count,1384617.00,1384617.00
mean,1706297.62,25556.24
std,989732.65,14121.27
min,1.00,1.00
25%,843370.00,13380.00
50%,1701880.00,25298.00
75%,2568023.00,37940.00
max,3421070.00,49688.00


In [7]:
# rozkład produktów
data['product_name'].value_counts()

Banana                                                  18726
Bag of Organic Bananas                                  15480
Organic Strawberries                                    10894
Organic Baby Spinach                                     9784
Large Lemon                                              8135
                                                        ...  
Automatic Air Freshener Spray Refill Hawaiian Breeze        1
Juniper Berries                                             1
PM Simply Sleep Nighttime Sleep Aid Caplets                 1
Prunes, Pitted, Bite Size                                   1
Gluten Free Buttermilk Pancake Mix                          1
Name: product_name, Length: 39123, dtype: int64

In [8]:
# liczba transakcji unikalnych
data['order_id'].nunique()

131209

In [9]:
# produkty łączy przecinkiem
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

order_id
1          Organic Whole String Cheese,Organic Celery Hea...
36         Super Greens Salad,Spring Water,Organic Garnet...
38         Green Peas,Organic Baby Arugula,Shelled Pistac...
96         Organic Raspberries,Organic Pomegranate Kernel...
98         Uncured Applewood Smoked Bacon,Black Beans,Org...
                                 ...                        
3421049    Gluten Free Rice Bread,Organic Baby Broccoli,O...
3421056    Sparkling Lemon Water,Brioche Buns,Homestyle C...
3421058    Classic Britannia Crisps,Club Soda Lower Sodiu...
3421063    Natural Artesian Water,Organic Half & Half,Twi...
3421070    Broccoli Florettes,Organic Unsweetened Almond ...
Name: product_name, Length: 131209, dtype: object

In [10]:
transactions = transactions.str.split(',')
transactions

order_id
1          [Organic Whole String Cheese, Organic Celery H...
36         [Super Greens Salad, Spring Water, Organic Gar...
38         [Green Peas, Organic Baby Arugula, Shelled Pis...
96         [Organic Raspberries, Organic Pomegranate Kern...
98         [Uncured Applewood Smoked Bacon, Black Beans, ...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Baby Broccoli...
3421056    [Sparkling Lemon Water, Brioche Buns, Homestyl...
3421058    [Classic Britannia Crisps, Club Soda Lower Sod...
3421063    [Natural Artesian Water, Organic Half & Half, ...
3421070    [Broccoli Florettes, Organic Unsweetened Almon...
Name: product_name, Length: 131209, dtype: object

### <a name='3'></a> Kodowanie transakcji

In [11]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
# zwraca macierz rzadką więc nie podejrzę danych
transactions_encoded = encoder.transform(transactions, sparse=True)
transactions_encoded

<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [12]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df
# mało wartości True

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,& Baby Wipes,& Blueberry with Quinoa Organic Baby Food,& Cheese Biscuit,& Cheese Croissant,& Cheese English Muffin,& Cheese Sandwiches,& Cheese Sauce,& Cheese Scramble,& Coconut With Red Lentils Organic Baby Food,& Garlic Pasta Sauce,& Grape Ice Pops,& Kale Ravioli,& Peas,& Pineapple,& Raisin,& Seasoning,& Sliced,& Yellow Peppers,& orange,0,0.87 oz) Air Care,004 Dark Brown,1,1 Ply,1 Subject,1 mg,1 to 1,1% Lowfat,1% Milkfat,1-1/2% Milkfat,...,for Tots Apple White Grape Juice,for Women Maximum Absorbency L Underwear,from Concentrate Mango Nectar,fruitwater® Strawberry Kiwi Sparkling Water,gel hand wash sea minerals,gelato Coffee Toffee,go fresh Cool Moisture Beauty,iChef Casserole Pans with Lids (10 7/16 in x 8 in x 1 3/4 in),in 100% Juice Mixed Fruit,in Gravy with Carrots Peas & Corn Mashed Potatoes & Meatloaf Nuggets,kattle Boiled & Hearth Baked Slice,o.b Super Plus Fluid Lock Tampons,of Hanover 100 Calorie Pretzels Mini,of Norwich Original English Mustard Powder Double Superfine,pumpkin spice,rich kiss Olive & Aloe Moisturizer 2 in 1,smart Blend Chicken & Rice Formula Dry Dog Food,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,w/Banana Pulp Free Juice,with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Bleach Powder Cleanser,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Pump Rebalancing Shampoo,with Seasoned Roasted Potatoes Scrambled Eggs & Sausage,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Sweet Cinnamon Bunches Cereal,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

### <a name='4'></a> Algorytm Apriori

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True, n_jobs=-1)
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

,support,itemsets
8,0.14,(Banana)
7,0.12,(Bag of Organic Bananas)
76,0.08,(Organic Strawberries)
41,0.07,(Organic Baby Spinach)
31,0.06,(Large Lemon)
37,0.06,(Organic Avocado)
61,0.06,(Organic Hass Avocado)
100,0.05,(Strawberries)
33,0.05,(Limes)
69,0.04,(Organic Raspberries)


In [14]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
# wybrał żeby zwracało tylko niektóre kolumny bo jest ich dużo
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
27,(Clementines),( Bag),0.01,0.52,36.84
26,( Bag),(Clementines),0.01,0.79,36.84
23,(Limes),(Large Lemon),0.01,0.26,4.26
22,(Large Lemon),(Limes),0.01,0.20,4.26
18,(Organic Raspberries),(Organic Strawberries),0.01,0.30,3.63
19,(Organic Strawberries),(Organic Raspberries),0.01,0.15,3.63
31,(Organic Avocado),(Large Lemon),0.01,0.18,2.94
30,(Large Lemon),(Organic Avocado),0.01,0.17,2.94
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.02,0.33,2.81
3,(Bag of Organic Bananas),(Organic Hass Avocado),0.02,0.16,2.81
